In [1]:
import os
import duckdb

# Show where the notebook thinks it is
print("Working directory:", os.getcwd())

csv_path = "Most-Recent-Cohorts-Institution.csv"
db_path = "ipeds.duckdb"
table_name = "institutions"

Working directory: c:\Users\mackm\Documents\School\SPR-26\CS 541\M1


In [2]:
assert os.path.exists(csv_path), f"CSV not found at: {csv_path}"
print("CSV size (MB):", round(os.path.getsize(csv_path) / (1024**2), 2))


CSV size (MB): 98.34


In [3]:
con = duckdb.connect(db_path)  # creates file if missing
print(con.execute("PRAGMA database_list;").df())


   seq   name                                               file
0  592  ipeds  c:\users\mackm\documents\school\spr-26\cs 541\...


In [4]:
con.execute(f"""
    CREATE OR REPLACE TABLE {table_name} AS
    SELECT *
    FROM read_csv_auto(?, header=True)
""", [csv_path])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
print(con.execute(f"SELECT COUNT(*) AS n FROM {table_name};").fetchone())
print(con.execute(f"SELECT * FROM {table_name} LIMIT 5;").df())
print(con.execute(f"DESCRIBE {table_name};").df())


(6429,)
   UNITID     OPEID  OPEID6                               INSTNM        CITY  \
0  100654  00100200  001002             Alabama A & M University      Normal   
1  100663  00105200  001052  University of Alabama at Birmingham  Birmingham   
2  100690  02503400  025034                   Amridge University  Montgomery   
3  100706  00105500  001055  University of Alabama in Huntsville  Huntsville   
4  100724  00100500  001005             Alabama State University  Montgomery   

  STABBR         ZIP                                       ACCREDAGENCY  \
0     AL       35762  Southern Association of Colleges and Schools C...   
1     AL  35294-0110  Southern Association of Colleges and Schools C...   
2     AL  36117-3553  Southern Association of Colleges and Schools C...   
3     AL       35899  Southern Association of Colleges and Schools C...   
4     AL  36104-0271  Southern Association of Colleges and Schools C...   

                              INSTURL  \
0                  

In [6]:
df = con.execute(f"""
    SELECT STABBR, COUNT(*) AS n
    FROM {table_name}
    GROUP BY STABBR
    ORDER BY n DESC
""").df()

df.head(10)


,STABBR,n
0,CA,671
1,NY,428
2,TX,415
3,FL,393
4,PA,331
5,OH,286
6,IL,245
7,NC,175
8,MI,173
9,GA,167


In [7]:
# close the connection and cleanup
con.close()
